<a href="https://colab.research.google.com/github/prathwishshetty/DS5500_HW_3/blob/master/DS5500_HW_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1>DS 5500 Homework 3 - Due Nov. 11</h1></center>

##Instructions  
<p>  
Create a new public Github repository for this homework assignment. The repository should include all of
the code necessary to reproduce your submitted solutions. Do not include the raw data in the repository.
</p>

*Use the README.md of the Github repository to present your solutions. Answer all questions completely for
full credit, including figures and tables where appropriate. For each problem, either provide relevant inline
code snippets, or cite the source file where the relevant code lives (with line numbers if appropriate).*

Describe any data processing steps (transformation, filtering, etc.) you perform when solving each problem,
providing reasoning where appropriate. You may need to be creative when deciding how to approach each
problem, as there may not be a single “correct” solution.

Your solutions should be posted as a **public** *note* on Piazza in the hw3 folder with the title “[hw3] - your name name”. Include in the body of the note a link to the Github repository with your solutions.

##Overview

In this homework assignment, you are a data scientist working for the U.S. federal government. Due to
budget cuts for education, your office has been tasked with cutting federal funding to some number of school
districts. Your supervisor has asked you to develop a recommendation and objective justification for their
decision using data analysis.

Due to related budget cuts, the most recent fiscal data you have to work with is from 2015-16. Download the
2015-16 district-level fiscal data from the National Center for Education Statistics’ Common Core of Data:

*   https://nces.ed.gov/ccd/f33agency.asp

For helping you make your decision, it may be helpful to have some performance metrics for each district.
You can download the 2015-16 data for distrct-level statistics on graduation rate and state assessments on
mathematics and reading/language arts from the EDFacts website:
* https://www2.ed.gov/about/inits/ed/edfacts/data-files/index.html  

These datasets can be linked based on the LEA IDs

### Problem 1

Import and explore the district-level fiscal data from 2015-16.

Rank and visualize the states that take in the most federal funding (revenue).

Which states spend the most federal funding per student?

In [0]:
#@title Download data

!wget https://nces.ed.gov/ccd/data/zip/sdf16_1a_sas7bdat.zip
!unzip sdf16_1a_sas7bdat.zip

In [0]:
import pandas as pd
import numpy as np

data = pd.read_sas('sdf16_1a.sas7bdat', format = 'sas7bdat', encoding="utf-8")

In [26]:
data.head(1)

,LEAID,CENSUSID,FIPST,CONUM,CSA,CBSA,NAME,STNAME,STABBR,SCHLEV,AGCHRT,YEAR,CCDNF,CENFILE,GSLO,GSHI,V33,MEMBERSCH,TOTALREV,TFEDREV,C14,C15,C16,C17,C19,B11,C20,C25,C36,B10,B12,B13,TSTREV,C01,C04,C05,C06,C07,C08,C09,...,FL_Z32,FL_Z33,FL_Z35,FL_Z36,FL_Z37,FL_Z38,FL_V11,FL_V13,FL_V15,FL_V17,FL_V21,FL_V23,FL_V37,FL_V29,FL_Z34,FL_V10,FL_V12,FL_V14,FL_V16,FL_V18,FL_V22,FL_V24,FL_V38,FL_V30,FL_V32,FL_V93,FL_19H,FL_21F,FL_31F,FL_41F,FL_61V,FL_66V,FL_W01,FL_W31,FL_W61,FL_V95,FL_V02,FL_K14,FL_CE1,FL_CE2
0,0100002,N,01,01073,142,13820,Alabama Youth Services,Alabama,AL,N,3,16,1,0,03,12,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M,M


In [46]:
data_state_revenue = data.groupby(['STNAME'])['TFEDREV'].sum().reset_index()
import plotly.express as px
fig = px.bar(data_state_revenue, x='STNAME', y='TFEDREV')
fig.show()

In [53]:
data_grouped=  data.groupby(['STNAME'])['TFEDREV','MEMBERSCH'].sum().reset_index()
data_grouped['rev_per_stud'] = data_grouped['TFEDREV']/data_grouped['MEMBERSCH']

,STNAME,TFEDREV,MEMBERSCH,rev_per_stud
0,Alabama,8.039069e+08,698220.0,1151.366211
1,Alaska,3.073200e+08,132477.0,2319.798908
2,Arizona,1.302010e+09,1092296.0,1191.993674
3,Arkansas,6.069460e+08,492085.0,1233.416936
4,California,7.709275e+09,6194722.0,1244.490842
5,Colorado,7.217189e+08,898845.0,802.940292
6,Connecticut,4.841860e+08,537821.0,900.273481
7,Delaware,1.447070e+08,134296.0,1077.522748
8,District of Columbia,2.262020e+08,82586.0,2738.987056
9,Florida,3.147329e+09,2791364.0,1127.523672


In [54]:
import plotly.express as px
fig = px.bar(data_grouped, x='STNAME', y='rev_per_stud')
fig.show()

### Problem 2

Visualize the relationship between school districts’ total revenue and expenditures.

Which states have the most debt per student?

### Problem 3

The district-level performance metrics from EDFacts may be useful in your decision.

However, to protect student privacy, the data in these datasets has been heavily “blurred” to prevent students
from being identified. Therefore, most of the numeric metrics are presented as ranges in string format. In
addition, censored and missing data must be imputed.

Write and explain a function for processing a single column of “blurred” metrics into usable numeric values.

Use it to process and then visualize the distribution of a performance metric of your choice.


###Problem 4

You are tasked with cutting 15% of the U.S. federal budget currently being spent on funding school districts.

How much money is this?

Choose which school districts will have their funding cut and how this will be done.

(You should produce a table of LEA IDs and the dollar amount by which their federal funding will be cut –
you do not need print the entire table.)


###Problem 5
Provide a statement for your supervisor justifying your decisions on which school districts will lose funding.